# Week 4: Halving the Surface Area of a Mesh
**Target**: Find the decision boundary of halving the surface area of a mesh

## Table of Contents
- [0 - Packages and Resources](#0)
- [1 - Implementation](#1)
    - [*1.0 - The Example](#1-0)
    - [1.1 - Area Calculation](#1-1)
    - [1.2 - BFS](#1-2)
    - [1.3 - Fit a Plane](#1-3)
    - [1.4 - Intersection of Plane and Mesh](#1-4)
- [2 - Sample Outcome](#2)
    - [2.1 - One Round](#2-1)
    - [2.2 - Convergence](#2-2)
- [3 - Reference](#3)

<a name='0'></a>
## 0. Packages and Resources

In [1]:
# Packages
import numpy as np
import random
import matplotlib.pyplot as plt

# Self-defined functions
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from util import util

# Visualization
import pyvista as pv
from pyvista import examples

<a name='1'></a>
## 1. Implementation

<a name='1-0'></a>
### *1.0 The Example

Here I use the sphere as an example.

In [2]:
sphere = pv.Sphere()

pl = pv.Plotter()
pl.add_mesh(sphere, show_edges=True, color="white")
pl.add_points(sphere.points, color="red", point_size=5)
pl.show()

Widget(value="<iframe src='http://localhost:53737/index.html?ui=P_0x2026c955e50_0&reconnect=auto' style='width…

<a name='1-1'></a>
### 1.1 Area Calculation

This is the preprocession part of a given mesh. I calculate the area of all triangles and store it in a array attached to the faces of a mesh.

In [3]:
sphere = sphere.compute_cell_sizes(area=True, length=False, volume=False)

In [4]:
sphere

PolyData (0x202718110c0)
  N Cells:    1680
  N Points:   842
  N Strips:   0
  X Bounds:   -4.993e-01, 4.993e-01
  Y Bounds:   -4.965e-01, 4.965e-01
  Z Bounds:   -5.000e-01, 5.000e-01
  N Arrays:   2

<a name='1-1'></a>
### 1.2 BFS

I apply BFS to find the initial decision boundary along the edges of the triangular mesh. The terminating condition is when the sum of area of chosen triangles is greater than our desired ratio. The algorithm start from a specified vertex.

#### 1.2.1 Select the Neighbourhood Faces

In [5]:
def find_faces_with_node(index, mesh):
    faces = mesh.faces.reshape((-1,4))[:, 1:4]
    return [i for i, face in enumerate(faces) if index in face]

In [6]:
# Randomly choose a vertex
test_target = random.randint(0, sphere.n_points-1)
print("The target vertex is: ", test_target)

neighbourhood_face = find_faces_with_node(test_target, sphere)

# Visualize
p1 = pv.Plotter()
p1.add_mesh(sphere, show_edges=True, color="white")
p1.add_points(sphere.points[test_target], color="blue")
p1.add_mesh(sphere.extract_cells(neighbourhood_face), show_edges=True, color="red")
p1.show()

The target vertex is:  546


Widget(value="<iframe src='http://localhost:53737/index.html?ui=P_0x20271834590_1&reconnect=auto' style='width…

#### 1.2.2 Select the Neighbourhood Vertex

Based on the function above, we can further specify the related vertices without repetition.

In [7]:
def find_connected_vertices(index, mesh):
    cids = find_faces_with_node(index, mesh)
    faces = mesh.faces.reshape((-1,4))[:, 1:4]
    connected = np.unique(faces[cids].ravel())
    return np.delete(connected, np.argwhere(connected == index))

In [8]:
connected_vertices = find_connected_vertices(test_target, sphere)

p2 = pv.Plotter()
p2.add_mesh(sphere, show_edges=True)
p2.add_mesh(sphere.points[test_target], color="blue", point_size=10)
p2.add_mesh(sphere.points[connected_vertices], color="red", point_size=10)
p2.show()

Widget(value="<iframe src='http://localhost:53737/index.html?ui=P_0x20271854590_2&reconnect=auto' style='width…

#### 1.2.3 The BFS Selecting Process

In [9]:
class Mesh_BFS_Area:
    def __init__(self, start, mesh):
        if not mesh.is_all_triangles:
            print("Not trianglar mesh! Fail!")
            
        self.mesh=mesh
        self.start=start
        
        self.mesh=self.mesh.compute_cell_sizes(length=False, volume=False)
        self.area=self.mesh.cell_data["Area"]
        self.surf_area=np.sum(self.mesh.cell_data["Area"])
        self.faces = mesh.faces.reshape((-1,4))[:, 1:4]
        
        self.vertex_queue = [start]
        self.face_queue = []
        self.selected_vertex = []
        self.selected_face = []
        
        self.current_area = 0
        self.err = 99999
    
    def reset(self):
        self.vertex_queue = []
        self.face_queue = []
        self.selected_vertex = []
        self.selected_face = []
        self.current_area = 0
        
    def one_round_test(self, ratio):
            target_area=self.surf_area * ratio

            current_vertex = self.vertex_queue.pop(0)
            self.find_connected_vertices(current_vertex)

            while(len(self.face_queue)!=0):
                current_face = self.face_queue.pop(0)
                self.selected_face.append(current_face)
                self.current_area += self.area[current_face]

                if(self.current_area > target_area):
                    break

            self.selected_vertex.append(current_vertex)
            
            
    def run(self, ratio):
        target_area=self.surf_area * ratio
        
        flag = True
        while(len(self.vertex_queue)!=0 and self.current_area < target_area):
            current_vertex = self.vertex_queue.pop(0)
            self.selected_vertex.append(current_vertex)
            self.find_connected_vertices(current_vertex)

            while(len(self.face_queue)!=0):
                current_face = self.face_queue.pop(0)  
                self.selected_face.append(current_face)
                self.current_area += self.area[current_face]

                if(self.current_area > target_area):
                    self.vertex_queue = [current_vertex] + self.vertex_queue
                    self.selected_vertex.pop()
                    self.err = self.current_area - target_area
#                     print(self.face_queue)
                    break
                                                                                                                                                                                                                                                                                                                                         
    
    def find_faces_with_node(self, target):
        new_faces = [i for i, face in enumerate(self.faces) if target in face and i not in self.selected_face]
        self.face_queue = self.face_queue + new_faces
        return new_faces
    
    def find_connected_vertices(self, target):
        new_faces = self.find_faces_with_node(target)
        vertices_id = np.unique(self.faces[new_faces].ravel())
        new_vertices = [i for i in vertices_id if i not in self.selected_vertex and i not in self.vertex_queue and i != target]
        self.vertex_queue = self.vertex_queue + new_vertices
        return new_vertices

In [10]:
test=Mesh_BFS_Area(200, sphere)

In [11]:
test.run(0.2)

In [12]:
plane, center, normal = pv.fit_plane_to_points(
    sphere.points[test.vertex_queue], return_meta=True
)

In [13]:
plane=plane.delaunay_2d()

In [14]:
plane.is_all_triangles

True <CallableBool>

In [15]:
intersection, s1_split, s2_split = sphere.intersection(sphere, split_first=False, split_second=True)

ERROR:root:Algorithm vtkIntersectionPolyDataFilter (000002027E45B380) returned failure for request: vtkInformation (000002027E044F60)


In [16]:
plane

PolyData (0x202718e1780)
  N Cells:    200
  N Points:   121
  N Strips:   0
  X Bounds:   -4.291e-01, 6.155e-01
  Y Bounds:   -4.376e-01, 5.032e-01
  Z Bounds:   -5.525e-01, -3.937e-02
  N Arrays:   2

In [17]:
intersection

PolyData (0x202718e2020)
  N Cells:    2380
  N Points:   2182
  N Strips:   0
  X Bounds:   -4.993e-01, 4.993e-01
  Y Bounds:   -4.999e-01, 4.999e-01
  Z Bounds:   -5.000e-01, 5.000e-01
  N Arrays:   3

In [20]:
rabbit = examples.download_bunny_coarse()
test2=Mesh_BFS_Area(200, rabbit)

In [23]:
p = pv.Plotter()
p.add_mesh(rabbit, show_edges=True)
p.add_mesh(rabbit.extract_cells(test.selected_face), color="red", show_edges=True)
# p.add_mesh(mesh.extract_cells(test.face_queue), color="green", show_edges=True)
p.add_mesh(rabbit.points[200], color="blue", point_size=10)
p.add_mesh(rabbit.points[test.vertex_queue], color="blue", point_size=10)
p.add_mesh(plane)
# p.add_mesh(intersection, color='yellow', line_width=10)
# p.add_mesh(intersection.points, color="green", point_size=10)
# p.add_mesh(mesh.points[191], color="green", point_size=10)
# p.add_mesh(mesh.points[], color="yellow", point_size=10)
# p.add_mesh(mesh.extract_cells(neighbour), color="yellow", show_edges=True)
# pl.camera_position = [(0.02, 0.30, 0.73), (0.02, 0.03, -0.022), (-0.03, 0.94, -0.34)]
p.show()

Widget(value="<iframe src='http://localhost:53737/index.html?ui=P_0x202053b1910_6&reconnect=auto' style='width…